# Stellarator Optimization

This tutorial demonstrates how to perform optimization with DESC. It will go through two examples: 1) targeting the "triple product" quasi-symmetry objective $f_T$ in full plasma volume 2) targeting the "two-term" objective $f_C$ for quasi-helical symmetry at the plasma boundary surface. These examples are adapted from a problem first presented in [Rodriguez et al (2022)](https://www.cambridge.org/core/journals/journal-of-plasma-physics/article/measures-of-quasisymmetry-for-stellarators/01B9DFE86A23964F331E0E0615B4E7A2) and reproduced in [Dudt et al (2022)](https://arxiv.org/abs/2204.00078). 

In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath('.'))
sys.path.append(os.path.abspath('../../'))

If you have acess to a GPU, uncomment the line `set_device("gpu")`: 

In [4]:
from desc import set_device
# set_device("gpu")

ModuleNotFoundError: No module named 'desc'

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import desc.io
from desc.grid import LinearGrid, ConcentricGrid
from desc.objectives import (
    ObjectiveFunction,
    FixBoundaryR,
    FixBoundaryZ,
    FixPressure,
    FixIota,
    FixPsi,
    ForceBalance,
    QuasisymmetryTwoTerm,
    QuasisymmetryTripleProduct,
)
from desc.optimize import Optimizer
from desc.plotting import plot_boozer_modes, plot_boozer_surface, plot_qs_error

ModuleNotFoundError: No module named 'desc'

Load an equilibrium solution that we will use for the initial guess for optimization. 

In [ ]:
eq_init = desc.io.load("qs_initial_guess.h5")

plot_boozer_modes(eq_init);
plot_boozer_surface(eq_init);
plot_qs_error(eq_init, fT=False, fC=False);

Explain with math what these quantities are

### Boozer Coordinates

\begin{equation}
|\mathbf{B}| = B(\psi, M\vartheta_{B} - N\zeta_{B})
\end{equation}

\begin{equation}
\mathbf{f}_{B} = \{ B_{mn} ~|~ m/n \neq M/N \}
\end{equation}

\begin{equation}
\hat{f}_{B} = \frac{|\mathbf{f}_{B}|}{\sqrt{\sum_{m,n} B_{mn}^2}}
\end{equation}

### Two-Term

\begin{equation}
f_{C} = \left( M \iota - N \right) \left( \mathbf{B} \times \nabla \psi \right) \cdot \nabla B - \left( M G + N I \right) \mathbf{B} \cdot \nabla B. 
\end{equation}

\begin{equation}
\mathbf{f}_{C} = \{ f_{C}(\theta_{i},\zeta_{j}) ~|~ i \in [0,2\pi), j \in [0,2\pi/N_{FP}) \}
\end{equation}

\begin{equation}
\hat{f}_{C} = \frac{\langle |f_C| \rangle}{\langle B \rangle^3}.
\end{equation}

### Triple Product

\begin{equation}
f_{T} = \nabla \psi \times \nabla B \cdot \nabla \left( \mathbf{B} \cdot \nabla B \right)
\end{equation}

\begin{equation}
\label{eq:fT_vec}
\mathbf{f}_{T} = \{ f_{T}(\theta_{i},\zeta_{j}) ~|~ i \in [0,2\pi), j \in [0,2\pi/N_{FP}) \}
\end{equation}

\begin{equation}
\label{eq:fT_hat}
\hat{f}_{T}(\psi) = \frac{\langle R \rangle^2 \langle |f_{T}| \rangle}{\langle B \rangle^4}
\end{equation}

This is the optimization routine we are going to use, but you could use others. 

In [6]:
optimizer = Optimizer("lsq-exact")

NameError: name 'Optimizer' is not defined

Specify the optimization constraints. In this case we only want 4 specific boundary modes to be the optimization variables, and we fix everything else. 

In [ ]:
# indices of boundary modes we want to optimize
idx_Rcc = eq_init.surface.R_basis.get_idx(M=1, N=2)
idx_Rss = eq_init.surface.R_basis.get_idx(M=-1, N=-2)
idx_Zsc = eq_init.surface.Z_basis.get_idx(M=-1, N=2)
idx_Zcs = eq_init.surface.Z_basis.get_idx(M=1, N=-2)

# boundary modes to constrain
R_modes = np.delete(eq_init.surface.R_basis.modes, [idx_Rcc, idx_Rss], axis=0)
Z_modes = np.delete(eq_init.surface.Z_basis.modes, [idx_Zsc, idx_Zcs], axis=0)

# constraints
constraints = (
    ForceBalance(),
    FixBoundaryR(modes=R_modes),
    FixBoundaryZ(modes=Z_modes),
    FixPressure(),
    FixIota(),
    FixPsi(),
)

## Triple Product QS in Volume

First we are going to optimize for f_T in a volume. 

In [ ]:
# objective
grid = ConcentricGrid(L=eq_init.L_grid, M=eq_init.M_grid, N=eq_init.N_grid, NFP=eq_init.NFP, sym=eq_init.sym)
objective_fT = ObjectiveFunction(QuasisymmetryTripleProduct(grid=grid), verbose=0)

Now run the optimization! 

In [ ]:
eq_qs_T, result_T = eq_init.optimize(
    objective=objective_fT,
    constraints=constraints,
    optimizer=optimizer,
    ftol=1e-2,
    xtol=1e-6,
    gtol=1e-6,
    maxiter=50,
    options={
        "perturb_options": {"order": 2, "verbose": 0},
        "solve_options": {"ftol": 1e-2, "xtol": 1e-6, "gtol": 1e-6, "verbose": 0},
    },
    copy=True,
    verbose=3,
)

Plot QS: 

In [ ]:
plot_boozer_modes(eq_qs_T);
plot_boozer_surface(eq_qs_T);

In [ ]:
# compare f_T & f_C before (o) vs after (x) optimization
fig, ax = plot_qs_error(eq_init, fB=False);
plot_qs_error(eq_qs_T, ax=ax, fB=False);
for line in plt.gca().lines[2:]:
    line.set_marker('x')

In [ ]:
# compare f_B before (o) vs after (x) optimization
fig, ax = plot_qs_error(eq_init, fT=False, fC=False);
plot_qs_error(eq_qs_T, ax=ax, fT=False, fC=False);
for line in plt.gca().lines[2:]:
    line.set_marker('x')

## Two-Term QH at Boundary Surface

Next we are going to optimize for f_T at the boundary surface. 

In [ ]:
grid = LinearGrid(M=2 * eq.M_grid + 1, N=2 * eq.N_grid + 1, NFP=eq.NFP, sym=eq.sym, rho=1.0)
objective_fC = ObjectiveFunction(QuasisymmetryTwoTerm(helicity=(1, eq.NFP)), verbose=0)

Now run the optimization! 

In [ ]:
eq_qs_C, result_C = eq_init.optimize(
    objective=objective_fC,
    constraints=constraints,
    optimizer=optimizer,
    ftol=1e-2,
    xtol=1e-6,
    gtol=1e-6,
    maxiter=50,
    options={
        "perturb_options": {"order": 2, "verbose": 0},
        "solve_options": {"ftol": 1e-2, "xtol": 1e-6, "gtol": 1e-6, "verbose": 0},
    },
    copy=True,
    verbose=3,
)

Plot QS: 

In [ ]:
plot_boozer_modes(eq_qs_C);
plot_boozer_surface(eq_qs_C);

In [ ]:
# compare f_T & f_C before (o) vs after (x) optimization
fig, ax = plot_qs_error(eq_init, fB=False);
plot_qs_error(eq_qs_C, ax=ax, fB=False);
for line in plt.gca().lines[2:]:
    line.set_marker('x')

In [ ]:
# compare f_B before (o) vs after (x) optimization
fig, ax = plot_qs_error(eq_init, fT=False, fC=False);
plot_qs_error(eq_qs_C, ax=ax, fT=False, fC=False);
for line in plt.gca().lines[2:]:
    line.set_marker('x')